In [3]:
import pandas as pd
import numpy as np
import scanpy
from scipy.spatial.distance import cosine
from sklearn.preprocessing import StandardScaler



In [18]:
pd.read_csv("~/Downloads/test_matrix.txt", sep="\t").T.to_csv('/Users/shayecarver/condPCA/final_method/test_matrix.txt', sep='\t', index=True)

In [27]:
pd.read_csv("/Users/shayecarver/condPCA/final_method/test_matrix.txt", sep="\t", index_col=0).T

,GTCATCCTCCACGGAC-18,ATTTACCAGTTTAGGA-18,AGTGACTAGCAGCCCT-13,CAACGGCAGATAGCAT-2,TGATTTCCACATGTTG-5,TGCTGAATCGTAGGAG-11,AGAAGTACACCCTGTT-2,GCCTGTTTCATCGCAA-15,TACTGCCGTTTACGAC-5,TTAATCCGTAGCGATG-4,...,TAGACTGTCTCACGAA-2,TGATGCACAAATGCTC-5,ACCTGAAAGGTATCTC-2,AGCTCAACAGACCGCT-11,TGTGATGAGTTGCCCG-15,TGACTCCCATGACAGG-11,GCAGCTGGTTCCACGG-12,CGTAATGTCGTCAAAC-4,TCGTCCAGTACCTAGT-1,CAGGGCTGTACAATAG-11
AC109466.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC008415.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC016687.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LINC02055,0,0,1,0,0,0,0,0,38,0,...,0,0,0,11,0,0,0,0,0,16
RELN,0,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RIPOR3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,14,0,0,0,0
MYO3A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC064875.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
STAMBPL1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,3,0,0,0,0,0,2


In [32]:
# check arguments
# check that no columns or rows with all zeros
#ensure that factor variables are NOT integers or real

class condPCA(object):
    def __init__(self, count_matrix_path, metadata_path, object_columns, vars_to_regress=True):
        """
        Parameters
        ----------
        count_matrix:
            Count matrix that must be QC'd

        metadata:
            metadata containing cell type labels named "celltype"

        object_columns:
            columns that will be one hot encoded/columns that are factors 

        vars_to_regress:
            list of variables to regress out

        """
        self.count_matrix = scanpy.read(count_matrix_path) # cells x genes, pd.read_csv(count_matrix_path, sep='\t', header=0, index_col=0)
        self.metadata = pd.read_csv(metadata_path, sep='\t', header=0, index_col=0)
        if vars_to_regress:
            self.vars_to_regress = self.metadata.columns
        else: # if vars_to_regress is a list, convert to pandas core Index object
            self.vars_to_regress = pd.Index(vars_to_regress)

        # one hot encode necessary metadata variables
        self.object_columns = object_columns # obtain columns that must be one hot encoded
        self.metadata[self.object_columns] = self.metadata[self.object_columns].astype(object) # convert these columns to objects

    def Normalize(self):
        """ 
        Normalize and take log1p of count data
        """
        scanpy.pp.normalize_total(self.count_matrix, target_sum = 10000) # update scanpy object to normalize all rows, so every cell sums to 10k
        scanpy.pp.log1p(self.count_matrix)

    def Standardize(self):
        """ 
        Standardize count data AND metadata
        """
        # Standardize count data
        mean_count = np.mean(self.count_matrix.X, axis=0)
        std_count = np.std(self.count_matrix.X, axis=0)
        self.standardized_count_data = (self.count_matrix.X - mean_count) / std_count # standardize


        # Standardize metadata
        self.metadata = self.metadata[self.vars_to_regress] # subset to only variables that you want to regress out
        # WARNING IN FOLLOWING LINE BECAUSE CONVERTING OBJECT THAT LOOKS NUMERIC TO BE ONE HOT ENCODED, this is batch
        self.metadata = pd.get_dummies(self.metadata, drop_first=True) # Convert factor covariates to dummy variables dropping one column 
        mean = np.mean(self.metadata, axis=0) # compute mean of each column of metadata
        std = np.std(self.metadata, axis=0) # compute standard deviation of each column of metadata

        self.standardized_metadata = (self.metadata - mean) / std # standardize
        

    #def fit(self):
        #self.standardized_data = np.c_[np.ones((self.standardized_data.shape[0], 1)), self.standardized_data] # append ones to standardized meta for intercept
        

In [33]:
# instantiate class
test = condPCA(count_matrix_path="/Users/shayecarver/condPCA/final_method/test_matrix.txt", metadata_path="/Users/shayecarver/condPCA/final_method/test_metadata.txt", object_columns=['Batch', 'Sex','celltype'])
test.Normalize()
test.Standardize()

/var/folders/1b/d3mmyllj1jg4btvm4vqztkdh0000gp/T/ipykernel_1905/2027839843.py:54: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  self.metadata = pd.get_dummies(self.metadata, drop_first=True) # Convert factor covariates to dummy variables dropping one column
